In [5]:
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [6]:
root_path = './'
label_dict = {'dws': 0, 'ups': 1, 'wlk': 2, 'jog': 3, 'sit': 4, 'std': 5}

In [16]:
def process_file(file_path, multi=1):
    df = pd.read_csv(file_path, index_col=0)
    df = pd.concat([df.iloc[:,9:12], df.iloc[:,3:6]], axis=1)
    # print(df)
    data = df.to_numpy()
    # print(data.shape)
    # print(data)
    X = []
    if data.shape[0] < 500:
        print(f"file {file_path} is too short")
        return None
    for i in range(0, data.shape[0] - 500, 500):
        # print(f'i = {i}, multi = {multi}')
        X.append(data[i:i+500].T)
    # print([i.shape for i in X])
    return X

# process_file('./ups_3/sub_1.csv')

In [17]:
X_samelen = []
Y_samelen = []
for _, label_dirs, _ in os.walk(root_path):
    for label_dir in label_dirs:
        if "Preprocess" in label_dir:
            continue
        label = label_dict[label_dir.split("_")[0]]
        for _, __, files in os.walk(os.path.join(root_path, label_dir)):
            for file in files:
                file_path = os.path.join(root_path, label_dir, file)
                # print(file_path)
                X = process_file(file_path)
                if X is None:
                    continue
                Y = np.full((len(X)), label)
                X_samelen.extend(X)
                Y_samelen.extend(Y)

file ./ups_12/sub_13.csv is too short
file ./dws_11/sub_24.csv is too short


In [23]:
X_final = np.stack(X_samelen, axis=0)
Y_final = np.stack(Y_samelen, axis=0)
print(X_final.shape, Y_final.shape)
# print(X_final)
print(Y_final)

(2635, 6, 500) (2635,)
[1 1 1 ... 2 2 2]


In [24]:
output_path = './PreprocessedData/'
if not os.path.exists(output_path):
    os.makedirs(output_path)
# save X_final and Y_final as npy files
np.save(os.path.join(output_path, 'total_X.npy'), X_final)
np.save(os.path.join(output_path, 'total_Y.npy'), Y_final)

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, train_size=0.7, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

np.save(os.path.join(output_path, 'X_train.npy'), X_train)
np.save(os.path.join(output_path, 'X_test.npy'), X_test)
np.save(os.path.join(output_path, 'Y_train.npy'), Y_train)
np.save(os.path.join(output_path, 'Y_test.npy'), Y_test)

(1844, 6, 500)
(791, 6, 500)
(1844,)
(791,)


In [26]:
output_path = './PreprocessedData/'
# load X_final and Y_final from npy files
X_final_load = np.load(os.path.join(output_path, 'total_X.npy'))
Y_final_load = np.load(os.path.join(output_path, 'total_Y.npy'))
# check if X_final and X_final_load are the same
print(np.all(X_final == X_final_load))
# check if Y_final and Y_final_load are the same
print(np.all(Y_final == Y_final_load))

True
True


In [27]:
output_path = './PreprocessedData/'
X_train_load = np.load(os.path.join(output_path, 'X_train.npy'))
X_test_load = np.load(os.path.join(output_path, 'X_test.npy'))
Y_train_load = np.load(os.path.join(output_path, 'Y_train.npy'))
Y_test_load = np.load(os.path.join(output_path, 'Y_test.npy'))
# check if X_train and X_train_load are the same
print(np.all(X_train == X_train_load))
# check if X_test and X_test_load are the same
print(np.all(X_test == X_test_load))
# check if Y_train and Y_train_load are the same
print(np.all(Y_train == Y_train_load))
# check if Y_test and Y_test_load are the same
print(np.all(Y_test == Y_test_load))

True
True
True
True
